# Graph-level inputs and outputs

## Introduction

When constructing complex workflows, you may encounter tasks that share input parameters (e.g. `code`). Also, you may wish to reorder or rename internal task outputs at the top level (e.g. `wg.outputs.optimized_stuff` instead of `wg.outputs.optimize.stuff`).

`WorkGraph` allows you to define **graph-level inputs and outputs** to:

- **Share inputs** across multiple tasks in a graph.
- **Aggregate outputs** from internal tasks, optionally organizing or renaming them.
- **Simplify the interface** by exposing shared parameters to users (retaining the flexibility of internal task parameter definitions).

## Example: Using graph-level inputs and outputs


In [ ]:
from aiida_workgraph import WorkGraph, task
from aiida import load_profile

load_profile()


@task
def add(x, y):
    return x + y

# Create a new WorkGraph
wg = WorkGraph("test_graph_inputs_outputs")

# Define graph-level input (of type `any`)
wg.add_input("workgraph.any", "a")

# Add tasks using the graph-level input
wg.add_task(add, "add1", x=wg.inputs.a, y=3)
wg.add_task(add, "add2", x=wg.inputs.a, y=wg.tasks.add1.outputs.result)

# Define graph-level outputs by exposing selected task results
wg.outputs.sum1 = wg.tasks.add1.outputs.result
wg.outputs.sum2 = wg.tasks.add2.outputs.result
wg.to_html()

### Run the workgraph


In [13]:
# Set the graph-level input
wg.inputs.a = 1
wg.submit(wait=True)

# Print the graph-level outputs
print("Graph-level outputs:")
print("sum1:", wg.outputs.sum1.value)
print("sum2:", wg.outputs.sum2.value)

WorkGraph process created, PK: 8014
Process 8014 finished with state: FINISHED
Graph-level outputs:
sum1: uuid: 94da4ac8-c0da-44ac-a5ac-e92d762b9d0b (pk: 8019) value: 4
sum2: uuid: 480a9666-e412-43be-8bf6-d025ea3c7f67 (pk: 8024) value: 5


### Note regarding `add_input`

Graph-level inputs can also be defined as `wg.inputs = {"a": ...}` (see following section). However, the above approach allows you to provide an identifier (e.g. `workgraph.any`) to the input, which is used for validation and for clearer identification when visualizing the graph (see below).

## Nested graph-level inputs and outputs

Graph-level inputs and outputs can be **nested** allowing you to group related parameters and results.

In [28]:
from aiida_workgraph import WorkGraph
from aiida import orm

@task
def add_more(x, y, z):
    return x + y + z

wg = WorkGraph(name="test_nested_graph_inputs_outputs")

# Define graph-level inputs for `add` tasks
x = orm.Int(1)
y = orm.Int(2)

wg.inputs = {
    "add": {
        "x": x,
        "y": y,
    },
}

# Use the nested inputs in tasks
wg.add_task(add_more, name="add1", x=wg.inputs.add.x, y=wg.inputs.add.y, z=1)
wg.add_task(add_more, name="add2", x=wg.inputs.add.x, y=wg.inputs.add.y, z=2)

# Define graph-level outputs by collecting results from the tasks
wg.outputs.results = {}
wg.outputs.results.sum1 = wg.tasks.add1.outputs.result
wg.outputs.results.sum2 = wg.tasks.add2.outputs.result

wg.to_html()

### Run the workgraph

In [29]:
wg.run()
print("result: ", wg.outputs.results)

07/07/2025 04:18:12 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|continue_workgraph]: tasks ready to run: add1,add2
07/07/2025 04:18:12 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|update_task_state]: Task: add1, type: PyFunction, finished.
07/07/2025 04:18:12 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|continue_workgraph]: tasks ready to run: add2
07/07/2025 04:18:12 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|update_task_state]: Task: add2, type: PyFunction, finished.
07/07/2025 04:18:12 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
07/07/2025 04:18:13 PM <250709> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8072|WorkGraphEngine|finalize]: Fina

result:  TaskSocketNamespace(name='results', sockets=['sum1', 'sum2'])


When we inspect the outputs of the `WorkGraph` process, we see that `sum1` and `sum2` are grouped under the `results` output:

In [30]:
!verdi process show {wg.pk}

Property     Value
-----------  -------------------------------------------
type         WorkGraph<test_nested_graph_inputs_outputs>
state        Finished [0]
pk           8072
uuid         8610ef7b-27e2-4b8f-86b2-a8a443f74a07
label        test_nested_graph_inputs_outputs
description
ctime        2025-07-07 14:18:11.952922+00:00
mtime        2025-07-07 14:18:13.091706+00:00

Inputs                                 PK    Type
-------------------------------------  ----  ------
workgraph_data
    meta_sockets
        graph_inputs
            sockets
                add
                    sockets
                        x
                            property
                                value  8070  Int
                        y
                            property
                                value  8071  Int
    tasks
        add1
            inputs
                sockets
                    z
                        property
                            value      8068  Int
     

### Note regarding AiiDA types

When defining graph-level inputs, providing AiiDA `orm` types ensures that tasks sharing the `orm` input are findable via the `orm` input node. In the case of the example above, we can look up both `add` tasks by filtering our query using the `pk` of the `x` input node:

In [40]:
from aiida import orm

for add_task in wg.tasks:
    print(f"{add_task.name}: {add_task.pk}")

pks = (
    orm.QueryBuilder()
    .append(
        orm.Int,
        filters={"id": x.pk},
        tag="x",
    )
    .append(
        orm.CalcFunctionNode,
        with_incoming="x",
        project="pk",
    )
    .all(flat=True)
)

print("Query results:", pks)


add1: 8075
add2: 8079
Query results: [8075, 8079]


### Automatic serialization of raw Python data

You can also pass raw Python values directly as inputs:

```python
wg.inputs = {
    "add_more": {
        "x": 1,
        "y": 2,
    },
}
```

In this case, `WorkGraph` will automatically serialize the raw Python data into the corresponding AiiDA Data nodes (e.g., an `int` becomes `orm.Int`, a `str` becomes `orm.Str`, etc.) before execution.
The exact serialization logic and all supported types (and how to register your own custom serializers) are described in detail in the **Data Serialization** section.

## Summary

In this section, you learned how to:

- Use `wg.add_input(...)` to define a shareable graph-level input
- Yse `wg.inputs = {...}` to define many inputs at once, or to define nested (namespaced) inputs to group related parameters
- Use graph-level inputs in tasks (`wg.inputs.<name>`)
- Use `wg.outputs.<name>` to expose relevant outputs from tasks
